In [37]:
import pandas as pd
import numpy as np
from datetime import date, datetime

In [10]:
df = pd.read_csv("data/faturamento_consolidade_com_feriado_e_receita_da_categoria.csv", parse_dates=["datetime"])
df.head()

,cod_loja,datetime,receita,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,...,ano,mes,categoria_loja,receita_trim1,receita_trim2,receita_trim3,receita_media,receita_ano_passado,cnt_feriados,receita_minha_categoria
0,1,2016-01-01,441833.37,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2016,1,40,0.0,0.00,0.0,0.000000,0.00,1.0,552910.464333
1,1,2017-01-01,404108.77,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2017,1,40,611797.3,876921.63,1257613.0,915443.976667,441833.37,1.0,552910.464333
2,1,2016-02-01,395340.22,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2016,2,40,0.0,0.00,0.0,0.000000,0.00,0.0,435209.493667
3,1,2017-02-01,483610.18,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2017,2,40,611797.3,876921.63,1257613.0,915443.976667,395340.22,0.0,435209.493667
4,1,2016-03-01,587810.18,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2016,3,40,0.0,0.00,0.0,0.000000,0.00,0.0,520654.621333


In [60]:
def get_mean_of_my_category(row):
    my_category_and_month = df[(df["categoria_loja"] == row["categoria_loja"]) & (df["mes"] == row["mes"])]
    
    return np.mean(my_category_and_month["receita"].values)

In [137]:
loja_gb = df.groupby("cod_loja")
i = 0

df_master = None
for cod_loja, loja_gb_df in loja_gb:
    lda_df = None
    filter_index = loja_gb_df.datetime.isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])
    ultimo_trimestre_2017 = loja_gb_df.loc[filter_index.values].copy().reset_index()
    
    #print(ultimo_trimestre_2017["receita_minha_categoria"])
    ### Cria um dataframe com 3 linhas para cada loja
    if ultimo_trimestre_2017.shape[0] == 3:
        ultimo_trimestre_2017 = pd.DataFrame(ultimo_trimestre_2017.sort_values("datetime"))
        
        trim1 = ultimo_trimestre_2017.iloc[0, :]["receita"]
        trim2 = ultimo_trimestre_2017.iloc[1, :]["receita"]
        trim3 = ultimo_trimestre_2017.iloc[2, :]["receita"]
        ultimo_trimestre_2017["receita_trim1"] = trim1
        ultimo_trimestre_2017["receita_trim2"] = trim2
        ultimo_trimestre_2017["receita_trim3"] = trim3
        ultimo_trimestre_2017["receita_media"] = np.mean([trim1, trim2, trim3])
        lda_df = ultimo_trimestre_2017
        
    elif ultimo_trimestre_2017.shape[0] == 0:
        v = df[df["cod_loja"] == cod_loja].iloc[0, :]
        
        valores = []
        for _ in range(3):
            valores.append(v.values.copy())
            
        lda_df = pd.DataFrame(valores, columns=df.columns.values)
        lda_df["receita_trim1"] = 0
        lda_df["receita_trim2"] = 0
        lda_df["receita_trim3"] = 0
        lda_df["receita_media"] = 0
        
    #elif ultimo_trimestre_2017.shape[0] < 3:
    else:
        valores = list(ultimo_trimestre_2017.values.copy())
        for _ in range(3 - ultimo_trimestre_2017.shape[0]):
            #print(ultimo_trimestre_2017.values)
            v = ultimo_trimestre_2017.values[0].copy()
            valores.append(v)
            
        lda_df = pd.DataFrame(valores, columns=ultimo_trimestre_2017.columns.values)
        lda_df["receita_trim1"] = 0
        lda_df["receita_trim2"] = 0
        lda_df["receita_trim3"] = 0
        lda_df["receita_media"] = 0

    lda_df.loc[0, "datetime"] = datetime(2018,1,1,0, 0, 0)
    lda_df.loc[1, "datetime"] = datetime(2018,2,1,0, 0, 0)     
    lda_df.loc[2, "datetime"] = datetime(2018,3,1,0, 0, 0)     
    lda_df["mes"] = lda_df["datetime"].dt.month
    lda_df["ano"] = lda_df["datetime"].dt.year
       
    ########## pegar a receita ano passo ############
    feriados_l = []
    receita_media_l = []
    for index, row in lda_df.iterrows():   
        year_ago = row["datetime"].replace(year=row["ano"]-1)
        year_agora_df = df[(df["cod_loja"] == row["cod_loja"]) & (df["datetime"] == year_ago)]
        feriados_df = df[(df["cod_loja"] == row["cod_loja"]) & (df["mes"] == row["mes"])]
        
      
        if year_agora_df.shape[0] == 1:
            receita_ano_passado = year_agora_df["receita"].values[0]
            receita_media_l.append(receita_ano_passado)
        else:
            receita_media_l.append(0)
            
        if feriados_df.shape[0] > 0:
            feriados_l.append(feriados_df["cnt_feriados"].values[0])
        else:
            feriados_l.append(0)
            
    #print(lda_df.apply(get_mean_of_my_category, axis=1))
    lda_df["cnt_feriados"] = feriados_l
    lda_df["receita_minha_categoria"] = receita_media_l
    lda_df["receita_minha_categoria"] = lda_df.apply(get_mean_of_my_category, axis=1)
    
    if df_master is None: df_master = lda_df
    else: df_master = pd.concat([df_master, lda_df])
        
#     i += 1
#     if i == 10: break
        
df_master.reset_index(drop=True).head(50)

,index,cod_loja,datetime,receita,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,...,ano,mes,categoria_loja,receita_trim1,receita_trim2,receita_trim3,receita_media,receita_ano_passado,cnt_feriados,receita_minha_categoria
0,19.0,1,2018-01-01,637339.76,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,...,2018,1,40,637339.76,891173.22,1216996.84,915169.940000,611797.30,1.0,552910.464333
1,21.0,1,2018-02-01,891173.22,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,...,2018,2,40,637339.76,891173.22,1216996.84,915169.940000,876921.63,0.0,435209.493667
2,23.0,1,2018-03-01,1216996.84,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,...,2018,3,40,637339.76,891173.22,1216996.84,915169.940000,1257613.00,0.0,520654.621333
3,43.0,2,2018-01-01,59996.08,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,...,2018,1,0,59996.08,76670.53,92232.32,76299.643333,53209.15,1.0,45986.626047
4,45.0,2,2018-02-01,76670.53,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,...,2018,2,0,59996.08,76670.53,92232.32,76299.643333,53830.14,0.0,34970.681963
5,47.0,2,2018-03-01,92232.32,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,...,2018,3,0,59996.08,76670.53,92232.32,76299.643333,108589.21,0.0,41821.363387
6,67.0,3,2018-01-01,143691.35,3550308005052,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,45.66,...,2018,1,10,143691.35,204050.94,420197.90,255980.063333,198425.47,1.0,107494.880422
7,69.0,3,2018-02-01,204050.94,3550308005052,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,45.66,...,2018,2,10,143691.35,204050.94,420197.90,255980.063333,199192.78,0.0,84367.133949
8,71.0,3,2018-03-01,420197.90,3550308005052,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,45.66,...,2018,3,10,143691.35,204050.94,420197.90,255980.063333,422565.37,0.0,101601.594107
9,91.0,4,2018-01-01,46064.16,3145604003002,3145604,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,30.00,...,2018,1,0,46064.16,64706.32,173683.70,94818.060000,41540.57,1.0,45986.626047


In [144]:
set(df_master["cod_loja"].values) - set(df["cod_loja"].values)


set()

In [146]:
df_master.drop("receita", axis=1).to_csv("")

,index,cod_loja,datetime,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,...,ano,mes,categoria_loja,receita_trim1,receita_trim2,receita_trim3,receita_media,receita_ano_passado,cnt_feriados,receita_minha_categoria
0,19.0,1,2018-01-01,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2018,1,40,637339.76,891173.22,1216996.84,915169.940000,611797.30,1.0,552910.464333
1,21.0,1,2018-02-01,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2018,2,40,637339.76,891173.22,1216996.84,915169.940000,876921.63,0.0,435209.493667
2,23.0,1,2018-03-01,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,1,21.76,FEATURE_05_VALUE_04,...,2018,3,40,637339.76,891173.22,1216996.84,915169.940000,1257613.00,0.0,520654.621333
0,43.0,2,2018-01-01,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,FEATURE_05_VALUE_03,...,2018,1,0,59996.08,76670.53,92232.32,76299.643333,53209.15,1.0,45986.626047
1,45.0,2,2018-02-01,5300108005004,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,1,22.00,FEATURE_05_VALUE_03,...,2018,2,0,59996.08,76670.53,92232.32,76299.643333,53830.14,0.0,34970.681963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,73617.0,3686,2018-02-01,3509700003001,3509700,FEATURE_01_VALUE_10,FEATURE_02_VALUE_06,0,31.76,FEATURE_05_VALUE_03,...,2018,2,10,88285.62,104081.23,275273.80,155880.216667,89173.66,0.0,84367.133949
2,73619.0,3686,2018-03-01,3509700003001,3509700,FEATURE_01_VALUE_10,FEATURE_02_VALUE_06,0,31.76,FEATURE_05_VALUE_03,...,2018,3,10,88285.62,104081.23,275273.80,155880.216667,255149.95,0.0,101601.594107
0,73639.0,3687,2018-01-01,4211702001001,4211702,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,26.50,FEATURE_05_VALUE_03,...,2018,1,0,61198.10,73877.02,186545.24,107206.786667,48056.18,1.0,45986.626047
1,73641.0,3687,2018-02-01,4211702001001,4211702,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,26.50,FEATURE_05_VALUE_03,...,2018,2,0,61198.10,73877.02,186545.24,107206.786667,63038.21,0.0,34970.681963
